Html Trading Journal


In [2]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, scrolledtext
from datetime import datetime, timedelta
import os
import shutil
import base64
import json
import requests
from pathlib import Path
import threading
import time

class QuantowerAPI:
    def __init__(self):
        # Note: Replace with actual Quantower API endpoints and authentication
        self.base_url = "http://localhost:8080/api"  # Default local API endpoint
        self.api_key = None
        self.session = requests.Session()
        
    def set_credentials(self, api_key=None, host="localhost", port="8080"):
        """Set API credentials and connection details"""
        self.api_key = api_key
        self.base_url = f"http://{host}:{port}/api"
        if api_key:
            self.session.headers.update({"Authorization": f"Bearer {api_key}"})
    
    def test_connection(self):
        """Test connection to Quantower API"""
        try:
            response = self.session.get(f"{self.base_url}/status", timeout=5)
            return response.status_code == 200
        except:
            return False
    
    def get_trades(self, start_date=None, end_date=None):
        """Fetch trades from Quantower API"""
        try:
            params = {}
            if start_date:
                params['start_date'] = start_date.isoformat()
            if end_date:
                params['end_date'] = end_date.isoformat()
            
            response = self.session.get(f"{self.base_url}/trades", params=params)
            if response.status_code == 200:
                return response.json()
            return None
        except Exception as e:
            print(f"Error fetching trades: {e}")
            return None
    
    def get_account_stats(self):
        """Fetch account statistics from Quantower API"""
        try:
            response = self.session.get(f"{self.base_url}/account/stats")
            if response.status_code == 200:
                return response.json()
            return None
        except Exception as e:
            print(f"Error fetching account stats: {e}")
            return None
    
    def get_positions(self):
        """Fetch current positions from Quantower API"""
        try:
            response = self.session.get(f"{self.base_url}/positions")
            if response.status_code == 200:
                return response.json()
            return None
        except Exception as e:
            print(f"Error fetching positions: {e}")
            return None

class WolfTradingJournal:
    def __init__(self, root):
        self.root = root
        self.root.title("Wolf's Trading Journal")
        self.root.geometry("1000x700")
        
        # Create journal directory on desktop
        desktop_path = Path.home() / "Desktop"
        self.journal_dir = desktop_path / "WolfTradingJournal"
        self.journal_dir.mkdir(exist_ok=True)
        self.images_dir = self.journal_dir / "images"
        self.images_dir.mkdir(exist_ok=True)
        
        self.html_file = self.journal_dir / "trading_journal.html"
        self.current_image_path = None
        
        # Initialize Quantower API
        self.quantower_api = QuantowerAPI()
        self.api_connected = False
        
        self.setup_ui()
        self.load_existing_journal()
        
    def setup_ui(self):
        # Create notebook for tabs
        notebook = ttk.Notebook(self.root)
        notebook.pack(fill="both", expand=True, padx=10, pady=10)
        
        # Manual Entry Tab
        self.manual_frame = ttk.Frame(notebook)
        notebook.add(self.manual_frame, text="Manual Entry")
        self.setup_manual_entry_tab()
        
        # API Connection Tab
        self.api_frame = ttk.Frame(notebook)
        notebook.add(self.api_frame, text="Quantower API")
        self.setup_api_tab()
        
        # Trade Import Tab
        self.import_frame = ttk.Frame(notebook)
        notebook.add(self.import_frame, text="Import Trades")
        self.setup_import_tab()
        
    def setup_manual_entry_tab(self):
        # Configure grid weights
        self.manual_frame.columnconfigure(1, weight=1)
        self.manual_frame.rowconfigure(3, weight=1)
        
        # Title
        title_label = ttk.Label(self.manual_frame, text="Wolf's Trading Journal", 
                               font=("Arial", 16, "bold"))
        title_label.grid(row=0, column=0, columnspan=3, pady=(10, 20))
        
        # Image upload section
        img_frame = ttk.LabelFrame(self.manual_frame, text="Screenshot/Chart Upload", padding="10")
        img_frame.grid(row=1, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(0, 10), padx=10)
        img_frame.columnconfigure(1, weight=1)
        
        ttk.Button(img_frame, text="Select Image", 
                  command=self.select_image).grid(row=0, column=0, padx=(0, 10))
        
        self.image_label = ttk.Label(img_frame, text="No image selected")
        self.image_label.grid(row=0, column=1, sticky=(tk.W, tk.E))
        
        # Entry type frame
        type_frame = ttk.LabelFrame(self.manual_frame, text="Entry Type", padding="10")
        type_frame.grid(row=2, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(0, 10), padx=10)
        
        self.entry_type = tk.StringVar(value="Trade Analysis")
        types = ["Trade Analysis", "Market Notes", "Strategy Review", "Daily Summary", "General Notes"]
        for i, entry_type in enumerate(types):
            ttk.Radiobutton(type_frame, text=entry_type, variable=self.entry_type, 
                           value=entry_type).grid(row=0, column=i, padx=10)
        
        # Notes section
        notes_frame = ttk.LabelFrame(self.manual_frame, text="Journal Entry", padding="10")
        notes_frame.grid(row=3, column=0, columnspan=3, sticky=(tk.W, tk.E, tk.N, tk.S), pady=(0, 10), padx=10)
        notes_frame.columnconfigure(0, weight=1)
        notes_frame.rowconfigure(2, weight=1)
        
        ttk.Label(notes_frame, text="Title:").grid(row=0, column=0, sticky=tk.W, pady=(0, 5))
        self.title_entry = ttk.Entry(notes_frame, font=("Arial", 12))
        self.title_entry.grid(row=1, column=0, sticky=(tk.W, tk.E), pady=(0, 10))
        
        ttk.Label(notes_frame, text="Notes:").grid(row=2, column=0, sticky=(tk.W, tk.N), pady=(0, 5))
        self.notes_text = scrolledtext.ScrolledText(notes_frame, wrap=tk.WORD, 
                                                   font=("Arial", 11), height=12)
        self.notes_text.grid(row=3, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # Buttons frame
        btn_frame = ttk.Frame(self.manual_frame)
        btn_frame.grid(row=4, column=0, columnspan=3, pady=(10, 10), padx=10)
        
        ttk.Button(btn_frame, text="Add Entry", 
                  command=self.add_manual_entry).grid(row=0, column=0, padx=(0, 10))
        ttk.Button(btn_frame, text="Clear Form", 
                  command=self.clear_form).grid(row=0, column=1, padx=(0, 10))
        ttk.Button(btn_frame, text="Open Journal", 
                  command=self.open_journal).grid(row=0, column=2, padx=(0, 10))
        ttk.Button(btn_frame, text="Show File Location", 
                  command=self.show_file_location).grid(row=0, column=3)
    
    def setup_api_tab(self):
        # API Connection Frame
        conn_frame = ttk.LabelFrame(self.api_frame, text="Quantower API Connection", padding="15")
        conn_frame.pack(fill="x", padx=10, pady=10)
        
        # Host and Port
        ttk.Label(conn_frame, text="Host:").grid(row=0, column=0, sticky="w", padx=(0, 5))
        self.host_entry = ttk.Entry(conn_frame, width=15)
        self.host_entry.insert(0, "localhost")
        self.host_entry.grid(row=0, column=1, padx=(0, 10))
        
        ttk.Label(conn_frame, text="Port:").grid(row=0, column=2, sticky="w", padx=(0, 5))
        self.port_entry = ttk.Entry(conn_frame, width=10)
        self.port_entry.insert(0, "8080")
        self.port_entry.grid(row=0, column=3, padx=(0, 10))
        
        # API Key
        ttk.Label(conn_frame, text="API Key (optional):").grid(row=1, column=0, sticky="w", pady=(10, 0), padx=(0, 5))
        self.api_key_entry = ttk.Entry(conn_frame, width=40, show="*")
        self.api_key_entry.grid(row=1, column=1, columnspan=2, sticky="ew", pady=(10, 0))
        
        # Connection buttons
        btn_frame = ttk.Frame(conn_frame)
        btn_frame.grid(row=2, column=0, columnspan=4, pady=(15, 0))
        
        self.connect_btn = ttk.Button(btn_frame, text="Test Connection", command=self.test_api_connection)
        self.connect_btn.pack(side="left", padx=(0, 10))
        
        self.connection_status = ttk.Label(btn_frame, text="Not Connected", foreground="red")
        self.connection_status.pack(side="left")
        
        # API Info Frame
        info_frame = ttk.LabelFrame(self.api_frame, text="Setup Instructions", padding="15")
        info_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        info_text = """To connect to Quantower API:

1. Enable API in Quantower platform:
   - Go to Quantower settings
   - Enable API access
   - Note the host and port (usually localhost:8080)

2. API Key (if required):
   - Generate API key in Quantower
   - Enter it in the field above

3. Test Connection:
   - Click "Test Connection" to verify setup
   - Status will show "Connected" when successful

4. Import Trades:
   - Use "Import Trades" tab to fetch your trading data
   - Trades will be automatically formatted for your journal

Note: This requires Quantower to be running with API enabled."""
        
        info_label = tk.Text(info_frame, wrap="word", height=15, font=("Arial", 10))
        info_label.insert("1.0", info_text)
        info_label.config(state="disabled")
        info_label.pack(fill="both", expand=True)
    
    def setup_import_tab(self):
        # Date Range Frame
        date_frame = ttk.LabelFrame(self.import_frame, text="Import Settings", padding="15")
        date_frame.pack(fill="x", padx=10, pady=10)
        
        ttk.Label(date_frame, text="Import trades from:").grid(row=0, column=0, sticky="w")
        
        self.date_range = tk.StringVar(value="today")
        ttk.Radiobutton(date_frame, text="Today", variable=self.date_range, 
                       value="today").grid(row=1, column=0, sticky="w", padx=(20, 0))
        ttk.Radiobutton(date_frame, text="Last 7 days", variable=self.date_range, 
                       value="week").grid(row=1, column=1, sticky="w", padx=(20, 0))
        ttk.Radiobutton(date_frame, text="Last 30 days", variable=self.date_range, 
                       value="month").grid(row=1, column=2, sticky="w", padx=(20, 0))
        
        # Import buttons
        btn_frame = ttk.Frame(date_frame)
        btn_frame.grid(row=2, column=0, columnspan=3, pady=(15, 0))
        
        ttk.Button(btn_frame, text="Import Trades", 
                  command=self.import_trades).pack(side="left", padx=(0, 10))
        ttk.Button(btn_frame, text="Import Account Stats", 
                  command=self.import_account_stats).pack(side="left", padx=(0, 10))
        ttk.Button(btn_frame, text="Import Positions", 
                  command=self.import_positions).pack(side="left")
        
        # Results Frame
        results_frame = ttk.LabelFrame(self.import_frame, text="Import Results", padding="15")
        results_frame.pack(fill="both", expand=True, padx=10, pady=10)
        
        self.results_text = scrolledtext.ScrolledText(results_frame, wrap=tk.WORD, 
                                                     font=("Courier", 10), height=20)
        self.results_text.pack(fill="both", expand=True)
    
    def test_api_connection(self):
        host = self.host_entry.get()
        port = self.port_entry.get()
        api_key = self.api_key_entry.get() if self.api_key_entry.get() else None
        
        self.quantower_api.set_credentials(api_key, host, port)
        
        if self.quantower_api.test_connection():
            self.connection_status.config(text="Connected", foreground="green")
            self.api_connected = True
            messagebox.showinfo("Success", "Successfully connected to Quantower API!")
        else:
            self.connection_status.config(text="Connection Failed", foreground="red")
            self.api_connected = False
            messagebox.showerror("Error", "Failed to connect to Quantower API.\n\nPlease check:\n- Quantower is running\n- API is enabled\n- Host/Port are correct")
    
    def import_trades(self):
        if not self.api_connected:
            messagebox.showwarning("Warning", "Please connect to Quantower API first!")
            return
        
        # Calculate date range
        end_date = datetime.now()
        if self.date_range.get() == "today":
            start_date = datetime.now().replace(hour=0, minute=0, second=0)
        elif self.date_range.get() == "week":
            start_date = end_date - timedelta(days=7)
        else:  # month
            start_date = end_date - timedelta(days=30)
        
        # Fetch trades in background
        def fetch_trades():
            trades = self.quantower_api.get_trades(start_date, end_date)
            if trades:
                self.root.after(0, lambda: self.process_imported_trades(trades))
            else:
                self.root.after(0, lambda: messagebox.showerror("Error", "Failed to fetch trades from API"))
        
        threading.Thread(target=fetch_trades, daemon=True).start()
        self.results_text.insert(tk.END, f"Fetching trades from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}...\n")
    
    def process_imported_trades(self, trades):
        if not trades:
            self.results_text.insert(tk.END, "No trades found for the selected period.\n")
            return
        
        # Format trades for journal entry
        trade_summary = f"=== IMPORTED TRADES ({len(trades)} trades) ===\n\n"
        
        total_pnl = 0
        winning_trades = 0
        
        for trade in trades:
            # Adjust these field names based on actual Quantower API response
            symbol = trade.get('symbol', 'N/A')
            side = trade.get('side', 'N/A')
            quantity = trade.get('quantity', 'N/A')
            entry_price = trade.get('entry_price', 'N/A')
            exit_price = trade.get('exit_price', 'N/A')
            pnl = trade.get('pnl', 0)
            timestamp = trade.get('timestamp', 'N/A')
            
            if pnl > 0:
                winning_trades += 1
            total_pnl += pnl
            
            trade_summary += f"• {symbol} - {side} {quantity} @ {entry_price} → {exit_price}\n"
            trade_summary += f"  P&L: ${pnl:.2f} | Time: {timestamp}\n\n"
        
        # Add summary statistics
        win_rate = (winning_trades / len(trades)) * 100 if trades else 0
        trade_summary += f"SUMMARY:\n"
        trade_summary += f"Total P&L: ${total_pnl:.2f}\n"
        trade_summary += f"Win Rate: {win_rate:.1f}% ({winning_trades}/{len(trades)})\n"
        trade_summary += f"Average per trade: ${total_pnl/len(trades):.2f}\n"
        
        # Create journal entry
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        entry_html = self.create_entry_html(
            f"Trading Session - {datetime.now().strftime('%Y-%m-%d')}", 
            trade_summary, 
            timestamp, 
            "Trade Analysis"
        )
        
        self.update_html_file(entry_html)
        
        # Update results display
        self.results_text.insert(tk.END, f"Successfully imported {len(trades)} trades!\n")
        self.results_text.insert(tk.END, f"Total P&L: ${total_pnl:.2f}\n")
        self.results_text.insert(tk.END, f"Win Rate: {win_rate:.1f}%\n\n")
        
        messagebox.showinfo("Success", f"Imported {len(trades)} trades to journal!")
    
    def import_account_stats(self):
        if not self.api_connected:
            messagebox.showwarning("Warning", "Please connect to Quantower API first!")
            return
        
        def fetch_stats():
            stats = self.quantower_api.get_account_stats()
            if stats:
                self.root.after(0, lambda: self.process_account_stats(stats))
            else:
                self.root.after(0, lambda: messagebox.showerror("Error", "Failed to fetch account stats"))
        
        threading.Thread(target=fetch_stats, daemon=True).start()
        self.results_text.insert(tk.END, "Fetching account statistics...\n")
    
    def process_account_stats(self, stats):
        # Format account stats for journal
        stats_summary = "=== ACCOUNT STATISTICS ===\n\n"
        
        # Adjust these field names based on actual Quantower API response
        balance = stats.get('balance', 'N/A')
        equity = stats.get('equity', 'N/A')
        margin = stats.get('margin_used', 'N/A')
        free_margin = stats.get('free_margin', 'N/A')
        
        stats_summary += f"Account Balance: ${balance}\n"
        stats_summary += f"Equity: ${equity}\n"
        stats_summary += f"Margin Used: ${margin}\n"
        stats_summary += f"Free Margin: ${free_margin}\n"
        
        # Create journal entry
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        entry_html = self.create_entry_html(
            f"Account Stats - {datetime.now().strftime('%Y-%m-%d')}", 
            stats_summary, 
            timestamp, 
            "Daily Summary"
        )
        
        self.update_html_file(entry_html)
        self.results_text.insert(tk.END, "Account statistics imported successfully!\n\n")
        messagebox.showinfo("Success", "Account statistics imported to journal!")
    
    def import_positions(self):
        if not self.api_connected:
            messagebox.showwarning("Warning", "Please connect to Quantower API first!")
            return
        
        def fetch_positions():
            positions = self.quantower_api.get_positions()
            if positions is not None:
                self.root.after(0, lambda: self.process_positions(positions))
            else:
                self.root.after(0, lambda: messagebox.showerror("Error", "Failed to fetch positions"))
        
        threading.Thread(target=fetch_positions, daemon=True).start()
        self.results_text.insert(tk.END, "Fetching current positions...\n")
    
    def process_positions(self, positions):
        if not positions:
            positions_summary = "=== CURRENT POSITIONS ===\n\nNo open positions.\n"
        else:
            positions_summary = f"=== CURRENT POSITIONS ({len(positions)} positions) ===\n\n"
            
            for pos in positions:
                symbol = pos.get('symbol', 'N/A')
                side = pos.get('side', 'N/A')
                size = pos.get('size', 'N/A')
                entry_price = pos.get('entry_price', 'N/A')
                current_price = pos.get('current_price', 'N/A')
                unrealized_pnl = pos.get('unrealized_pnl', 'N/A')
                
                positions_summary += f"• {symbol} - {side} {size}\n"
                positions_summary += f"  Entry: {entry_price} | Current: {current_price}\n"
                positions_summary += f"  Unrealized P&L: ${unrealized_pnl}\n\n"
        
        # Create journal entry
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        entry_html = self.create_entry_html(
            f"Open Positions - {datetime.now().strftime('%Y-%m-%d')}", 
            positions_summary, 
            timestamp, 
            "Market Notes"
        )
        
        self.update_html_file(entry_html)
        self.results_text.insert(tk.END, f"Current positions imported successfully!\n\n")
        messagebox.showinfo("Success", "Current positions imported to journal!")
    
    def select_image(self):
        file_types = [
            ("Image files", "*.jpg *.jpeg *.png *.gif *.bmp"),
            ("All files", "*.*")
        ]
        
        file_path = filedialog.askopenfilename(
            title="Select an image",
            filetypes=file_types
        )
        
        if file_path:
            self.current_image_path = file_path
            filename = os.path.basename(file_path)
            self.image_label.config(text=f"Selected: {filename}")
    
    def get_image_base64(self, image_path):
        """Convert image to base64 for embedding in HTML"""
        try:
            with open(image_path, "rb") as img_file:
                img_data = img_file.read()
                img_base64 = base64.b64encode(img_data).decode('utf-8')
                
                # Determine MIME type based on extension
                ext = os.path.splitext(image_path)[1].lower()
                mime_types = {
                    '.jpg': 'image/jpeg', '.jpeg': 'image/jpeg',
                    '.png': 'image/png', '.gif': 'image/gif',
                    '.bmp': 'image/bmp'
                }
                mime_type = mime_types.get(ext, 'image/jpeg')
                
                return f"data:{mime_type};base64,{img_base64}"
        except Exception as e:
            messagebox.showerror("Error", f"Could not process image: {str(e)}")
            return None
    
    def add_manual_entry(self):
        title = self.title_entry.get().strip()
        notes = self.notes_text.get("1.0", tk.END).strip()
        entry_type = self.entry_type.get()
        
        if not title and not notes:
            messagebox.showwarning("Warning", "Please enter a title or notes for your entry.")
            return
        
        # Create entry HTML
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        entry_html = self.create_entry_html(title, notes, timestamp, entry_type)
        
        # Update HTML file
        self.update_html_file(entry_html)
        
        # Clear form
        self.clear_form()
        
        messagebox.showinfo("Success", f"Entry added successfully!\nJournal saved to: {self.html_file}")
    
    def create_entry_html(self, title, notes, timestamp, entry_type="General Notes"):
        # Color coding based on entry type
        type_colors = {
            "Trade Analysis": "#4a90e2",
            "Market Notes": "#50c878",
            "Strategy Review": "#ff6b6b",
            "Daily Summary": "#ffa500",
            "General Notes": "#9b59b6"
        }
        
        border_color = type_colors.get(entry_type, "#4a90e2")
        
        entry_html = f"""
        <div class="entry" style="border-left-color: {border_color};">
            <div class="entry-header">
                <div>
                    <h2>{title if title else 'Untitled Entry'}</h2>
                    <span class="entry-type">{entry_type}</span>
                </div>
                <span class="timestamp">{timestamp}</span>
            </div>
        """
        
        if self.current_image_path:
            img_base64 = self.get_image_base64(self.current_image_path)
            if img_base64:
                entry_html += f"""
            <div class="image-container">
                <img src="{img_base64}" alt="Trading Chart/Screenshot" class="journal-image">
            </div>
                """
        
        if notes:
            # Convert line breaks to HTML and preserve formatting
            notes_html = notes.replace('\n', '<br>')
            entry_html += f"""
            <div class="notes">
                <p>{notes_html}</p>
            </div>
            """
        
        entry_html += """
        </div>
        """
        
        return entry_html
    
    def get_html_template(self):
        return """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Wolf's Trading Journal</title>
    <style>
        body {
            font-family: 'Georgia', serif;
            max-width: 900px;
            margin: 0 auto;
            padding: 20px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            line-height: 1.6;
        }
        
        .container {
            background: rgba(255, 255, 255, 0.95);
            border-radius: 15px;
            padding: 30px;
            box-shadow: 0 20px 40px rgba(0,0,0,0.1);
        }
        
        .header {
            text-align: center;
            border-bottom: 3px solid #333;
            padding-bottom: 20px;
            margin-bottom: 30px;
        }
        
        .header h1 {
            color: #333;
            font-size: 2.5em;
            margin: 0;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.1);
        }
        
        .header p {
            color: #666;
            font-style: italic;
            margin: 10px 0 0 0;
            font-size: 1.1em;
        }
        
        .entry {
            background: white;
            margin-bottom: 25px;
            padding: 25px;
            border-radius: 10px;
            box-shadow: 0 5px 15px rgba(0,0,0,0.08);
            border-left: 5px solid #4a90e2;
            transition: transform 0.2s ease;
        }
        
        .entry:hover {
            transform: translateY(-2px);
            box-shadow: 0 8px 25px rgba(0,0,0,0.12);
        }
        
        .entry-header {
            display: flex;
            justify-content: space-between;
            align-items: flex-start;
            margin-bottom: 20px;
            border-bottom: 1px solid #eee;
            padding-bottom: 15px;
        }
        
        .entry-header h2 {
            color: #333;
            margin: 0 0 5px 0;
            font-size: 1.4em;
        }
        
        .entry-type {
            background: #f8f9fa;
            color: #495057;
            padding: 3px 8px;
            border-radius: 12px;
            font-size: 0.8em;
            font-weight: 500;
            display: inline-block;
        }
        
        .timestamp {
            color: #888;
            font-size: 0.9em;
            font-style: italic;
            white-space: nowrap;
        }
        
        .image-container {
            margin: 20px 0;
            text-align: center;
        }
        
        .journal-image {
            max-width: 100%;
            height: auto;
            border-radius: 8px;
            box-shadow: 0 4px 12px rgba(0,0,0,0.15);
        }
        
        .notes {
            color: #444;
            font-size: 1.05em;
            line-height: 1.7;
        }
        
        .notes p {
            margin: 0;
        }
        
        .stats-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin: 20px 0;
        }
        
        .stat-box {
            background: #f8f9fa;
            padding: 15px;
            border-radius: 8px;
            text-align: center;
            border: 1px solid #e9ecef;
        }
        
        .stat-value {
            font-size: 1.5em;
            font-weight: bold;
            color: #333;
        }
        
        .stat-label {
            color: #666;
            font-size: 0.9em;
            margin-top: 5px;
        }
        
        .footer {
            text-align: center;
            margin-top: 50px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            color: #888;
            font-size: 0.9em;
        }
        
        @media (max-width: 600px) {
            body {
                padding: 10px;
            }
            
            .container {
                padding: 20px;
            }
            
            .entry-header {
                flex-direction: column;
                align-items: flex-start;
            }
            
            .entry-header h2 {
                margin-bottom: 10px;
            }
            
            .timestamp {
                margin-top: 5px;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>Wolf's Trading Journal</h1>
            <p>Tracking trades, strategies, and market insights</p>
        </div>
        
        <div id="entries">
            <!-- Journal entries will be inserted here -->
        </div>
        
        <div class="footer">
            <p>Generated with Wolf's Trading Journal</p>
        </div>
    </div>
</body>
</html>"""
    
    def load_existing_journal(self):
        if not self.html_file.exists():
            # Create initial HTML file
            with open(self.html_file, 'w', encoding='utf-8') as f:
                f.write(self.get_html_template())
    
    def update_html_file(self, new_entry_html):
        try:
            # Read existing HTML
            if self.html_file.exists():
                with open(self.html_file, 'r', encoding='utf-8') as f:
                    html_content = f.read()
            else:
                html_content = self.get_html_template()
            
            # Insert new entry at the beginning of entries section
            entries_marker = '<div id="entries">'
            if entries_marker in html_content:
                insertion_point = html_content.find(entries_marker) + len(entries_marker)
                updated_html = (html_content[:insertion_point] + 
                              new_entry_html + 
                              html_content[insertion_point:])
            else:
                # Fallback: insert before footer
                footer_marker = '<div class="footer">'
                insertion_point = html_content.find(footer_marker)
                updated_html = (html_content[:insertion_point] + 
                              new_entry_html + 
                              html_content[insertion_point:])
            
            # Write updated HTML
            with open(self.html_file, 'w', encoding='utf-8') as f:
                f.write(updated_html)
                
        except Exception as e:
            messagebox.showerror("Error", f"Could not update HTML file: {str(e)}")
    
    def clear_form(self):
        self.title_entry.delete(0, tk.END)
        self.notes_text.delete("1.0", tk.END)
        self.current_image_path = None
        self.image_label.config(text="No image selected")
    
    def open_journal(self):
        if self.html_file.exists():
            # Open HTML file in default browser
            os.startfile(str(self.html_file))
        else:
            messagebox.showinfo("Info", "No journal entries yet. Add an entry first!")
    
    def show_file_location(self):
        location_info = f"""Wolf's Trading Journal files are saved in:
        
Directory: {self.journal_dir}
HTML File: {self.html_file}
Images: {self.images_dir}

The HTML file updates automatically with each new entry.
Connect to Quantower API to automatically import trades!"""
        
        messagebox.showinfo("File Location", location_info)

def main():
    root = tk.Tk()
    app = WolfTradingJournal(root)
    root.mainloop()

if __name__ == "__main__":
    main()